# Cleaning and Validation

Political Alignment Case Study

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

This is the first in a series of notebooks that make up a [case study in exploratory data analysis](https://allendowney.github.io/PoliticalAlignmentCaseStudy/).
This case study is part of the [*Elements of Data Science*](https://allendowney.github.io/ElementsOfDataScience/) curriculum.

In this notebook, we 

1. Read data from the General Social Survey (GSS),

2. Clean the data, particularly dealing with special codes that indicate missing data,

3. Validate the data by comparing the values in the dataset with values documented in the codebook.

4. Generate resampled datasets that correct for deliberate oversampling in the dataset, and

5. Store the resampled data in a binary format (HDF5) that makes it easier to work with in the notebooks that follow this one.

The following cell loads the packages we need.  If you have everything installed, there should be no error messages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Reading the data

The data we'll use is from the General Social Survey (GSS).  Using the [GSS Data Explorer](https://gssdataexplorer.norc.org/projects/52787), I selected a subset of the variables in the GSS and made it available along with this notebook.
The following cell downloads this extract.

In [2]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/PoliticalAlignmentCaseStudy/' +
         'raw/master/gss_eda.tar.gz')

The data is in a gzipped tar file that contains two files:

* `GSS.dat` contains the actual data, and

* `GSS.dct` is the "dictionary file" that describes the contents of the data file.

The dictionary file is in Stata format, so we'll use the `statadict` library to read it.  The following cell installs it if necessary.

In [3]:
try:
    import statadict
except ImportError:
    !pip install statadict

We can use the Python module `tarfile` to extract the dictionary file, and `statadict` to read and parse it.

In [4]:
import tarfile
from statadict import parse_stata_dict

filename = 'gss_eda.tar.gz'
dict_file='GSS.dct'
data_file='GSS.dat'

with tarfile.open(filename) as tf:
    tf.extract(dict_file)
    stata_dict = parse_stata_dict(dict_file)
        
stata_dict

The result is a `StataDict` object that contains the names of the columns in the data file and the column specifications, which indicate where each column is.
We can pass these values to `read_fwf`, which is a Pandas function that reads [fixed width files](https://en.wikipedia.org/wiki/Flat-file_database), which is what `GSS.dat` is.

In [5]:
with tarfile.open(filename) as tf:
    fp = tf.extractfile(data_file)  
    gss = pd.read_fwf(fp,
                      names=stata_dict.names,
                      colspecs=stata_dict.colspecs)

The column names in the data file are in all caps.
I'll convert them to lower case because I think it makes the code look better.

In [6]:
gss.columns = gss.columns.str.lower()

We can use `shape` and `head` to see what the `DataFrame` looks like.

In [7]:
print(gss.shape)
gss.head()

(64814, 169)


,year,id_,agewed,divorce,sibs,childs,age,educ,paeduc,maeduc,...,fejobaff,discaffm,discaffw,fehire,meovrwrk,avoidbuy,income,rincome,realrinc,china
0,1972,1,0,0,3,0,23,16,10,97,...,0,0,0,0,0,0,0,0,0.0,-1
1,1972,2,21,2,4,5,70,10,8,8,...,0,0,0,0,0,0,0,0,0.0,-1
2,1972,3,20,2,5,4,48,12,8,8,...,0,0,0,0,0,0,0,0,0.0,-1
3,1972,4,24,2,5,0,27,17,16,12,...,0,0,0,0,0,0,0,0,0.0,-1
4,1972,5,22,2,2,2,61,12,8,8,...,0,0,0,0,0,0,0,0,0.0,-1


This dataset has 64814 rows, one for each respondent, and 166 columns, one for each variable.

## Validation

Now that we've got the data loaded, it is important to validate it, which means checking for errors.

The kinds of errors you have to check for depend on the nature of the data, the collection process, how the data is stored and transmitted, etc.

For this dataset, there are three kinds of validation we'll think about:

1) We need to check the **integrity** of the dataset; that is, whether the data were corrupted or changed during transmission, storage, or conversion from one format to another.

2) We need to check our **interpretation** of the data; for example, whether the numbers used to encode the data mean what we think they mean.

3) We will also keep an eye out for **invalid** data; for example, missing data might be represented using special codes, or there might be patterns in the data that indicate problems with the survey process and the recording of the data.  

In a different dataset I worked with, I found a surprising number of respondents whose height was supposedly 62 centimeters.  After investigating, I concluded that they were probably 6 feet, 2 inches, and their heights were recorded incorrectly.

Validating data can be a tedious process, but it is important.  If you interpret data incorrectly and publish invalid results, you will be embarrassed in the best case, and in the worst case you might do real harm.  See [this article](https://www.vox.com/future-perfect/2019/6/4/18650969/married-women-miserable-fake-paul-dolan-happiness) for a recent example.

However, I don't expect you to validate every variable in this dataset.  Instead, I will demonstrate the process, and then ask you to validate one additional variable as an exercise.

The first variable we'll validate is called `polviews`.  It records responses to the following question:

>We hear a lot of talk these days about liberals and conservatives. 
I'm going to show you a seven-point scale on which the political views that people might hold are arranged from extremely liberal--point 1--to extremely conservative--point 7. Where would you place yourself on this scale?

You can [read the documentation of this variable in the GSS codebook](https://gssdataexplorer.norc.org/projects/52787/variables/178/vshow).

The responses are encoded like this:

```
1	Extremely liberal
2	Liberal
3	Slightly liberal
4	Moderate
5	Slghtly conservative
6	Conservative
7	Extremely conservative
8	Don't know
9	No answer
0	Not applicable
```

The following function, `values`, takes a Series that represents a single variable and returns the values in the series and their frequencies.

In [8]:
def values(series):
    """Count the values and sort.
    
    series: pd.Series
    
    returns: series mapping from values to frequencies
    """
    return series.value_counts().sort_index()

Here are the values for the variable `polviews`.

In [9]:
column = gss['polviews']

In [10]:
values(column)

0     6777
1     1682
2     6514
3     7010
4    21370
5     8690
6     8230
7     1832
8     2326
9      383
Name: polviews, dtype: int64

To check the integrity of the data and confirm that we have loaded it correctly, we'll do a "spot check"; that is, we'll pick one year and compare the values we see in the dataset to the values reported in the codebook.

We can select values from a single year like this:

In [11]:
one_year = (gss['year'] == 1974)

And look at the values and their frequencies:

In [12]:
values(column[one_year])

1     22
2    201
3    207
4    564
5    221
6    160
7     35
8     70
9      4
Name: polviews, dtype: int64

If you [compare these results to the values in the codebook](https://gssdataexplorer.norc.org/projects/52787/variables/178/vshow), you should see that they agree.

**Exercise:** Go back and change 1974 to another year, and compare the results to the codebook.

## Missing data

For many variables, missing values are encoded with numerical codes that we need to replace before we do any analysis.

For `polviews`, the values 8, 9, and 0 represent "Don't know", "No answer", and "Not applicable".

"Not applicable" usually means the respondent was not asked a particular question.

To keep things simple, we'll treat all of these values as equivalent, but we lose some information by doing that.  For example, if a respondent refuses to answer a question, that might suggest something about their answer.  If so, treating their response as missing data might bias the results.

Fortunately, for most questions the number of respondents who refused to answer is small.

I'll replace the numeric codes 8, 9, and 0 with `NaN`, which is a special value used to indicate missing data. 

In [13]:
clean = column.replace([0, 8, 9], np.nan)

We can use `notna` and `sum` to count the valid responses:

In [14]:
clean.notna().sum()

55328

And we use `isna` to count the missing responses:

In [15]:
clean.isna().sum()

9486

We can [check these results against the codebook](https://gssdataexplorer.norc.org/projects/52787/variables/178/vshow); at the bottom of that page, it reports the number of "Valid cases" and "Missing cases".

However, in this example, the results don't match.  The codebook reports 53081 valid cases and 9385 missing cases.

To figure out what was wrong, I looked at the difference between the values in the codebook and the values I computed from the dataset.

In [16]:
clean.notna().sum() - 53081

2247

In [17]:
clean.isna().sum() - 9385

101

That looks like about one year of data, so I guessed that the numbers in the code book might not include the most recent data, from 2018.

Here are the numbers from 2018.

In [18]:
one_year = (gss['year'] == 2018)
one_year.sum()

2348

In [19]:
clean[one_year].notna().sum()

2247

In [20]:
clean[one_year].isna().sum()

101

It looks like my hypothesis is correct; the summary statistics in the codebook do not include data from 2018.

Based on these checks, it looks like the dataset is intact and we have loaded it correctly.

## Replacing missing data

For the other variables in this dataset, I read through the code book and identified the special values that indicate missing data.

I recorded that information in the following function, which is intended to replace special values with `NaN`.

In [21]:
def replace_invalid(df, columns, bad):
    for column in columns:
        df[column].replace(bad, np.nan, inplace=True)

def gss_replace_invalid(df):
    """Replace invalid data with NaN.
    
    df: DataFrame
    """
    # different variables use different codes for invalid data
    df.cohort.replace([0, 9999], np.nan, inplace=True)
    df.marcohrt.replace([0, 9999], np.nan, inplace=True)
        
    # since there are a lot of variables that use 0, 8, and 9 for invalid data,
    # I'll use a loop to replace all of them
    columns = ['abany', 'abdefect', 'abhlth', 'abnomore', 'abpoor', 'abrape', 
            'absingle', 'affrmact', 'bible', 'cappun', 'colath', 'colcom', 
            'colhomo', 'colmil', 'colmslm', 'colrac', 'compuse', 'conarmy', 
            'conbus', 'conclerg', 'coneduc', 'confed', 'confinan', 'conjudge', 
            'conlabor', 'conlegis', 'conmedic', 'conpress', 'consci', 'contv', 
            'databank', 'discaffm', 'discaffw', 'divlaw', 'divorce', 'fair', 
            'fear', 'fechld', 'fefam', 'fehire', 'fejobaff', 'fepol', 'fepresch', 
            'finrela', 'fund', 'fund16', 'god', 'grass', 'gunlaw', 'hapmar', 
            'happy', 'health', 'helpful', 'homosex', 'hunt', 'letdie1', 
            'libath', 'libcom', 'libhomo', 'libmil', 'libmslm', 'librac', 
            'life', 'memchurh', 'meovrwrk', 'nataid', 'natarms', 'natchld', 
            'natcity', 'natcrime', 'natcrimy', 'natdrug', 'nateduc', 'natenrgy', 
            'natenvir', 'natfare', 'natheal', 'natmass', 'natpark', 'natrace', 
            'natroad', 'natsci', 'natsoc', 'natspac', 'polviews', 'pornlaw', 
            'postlife', 'pray', 'prayer', 'premarsx', 'pres04', 'pres08', 'pres12', 
            'racmar', 'racopen', 'racpres', 'reborn', 'relexp', 'reliten', 
            'relpersn', 'res16', 'satfin', 'satjob', 'savesoul', 'sexeduc', 
            'spanking', 'spkath', 'spkcom', 'spkhomo', 'spkmil', 'spkmslm', 
            'spkrac', 'sprel16', 'sprtprsn', 'teensex', 'trust', 'union_', 'xmarsex']
    replace_invalid(df, columns, [0, 8, 9])

    columns = ['degree', 'padeg', 'madeg', 'spdeg', 'partyid']
    replace_invalid(df, columns, [8, 9])
        
    df.phone.replace([0, 2, 9], np.nan, inplace=True)
    df.owngun.replace([0, 3, 8, 9], np.nan, inplace=True)
    df.pistol.replace([0, 3, 8, 9], np.nan, inplace=True)
    df.class_.replace([0, 5, 8, 9], np.nan, inplace=True)
    
    df.chldidel.replace([-1, 8, 9], np.nan, inplace=True)

    df.attend.replace([9], np.nan, inplace=True)
    df.childs.replace([9], np.nan, inplace=True)
    df.adults.replace([9], np.nan, inplace=True)

    df.relactiv.replace([0, 98, 89], np.nan, inplace=True)
    df.age.replace([0, 98, 99], np.nan, inplace=True)
    df.agewed.replace([0, 98, 99], np.nan, inplace=True)
    df.relig.replace([0, 98, 99], np.nan, inplace=True)
    df.relig16.replace([0, 98, 99], np.nan, inplace=True)
    
    df.realinc.replace([0], np.nan, inplace=True)                  
    df.realrinc.replace([0], np.nan, inplace=True)                  
    
    # note: sibs contains some unlikely numbers
    df.sibs.replace([-1, 98, 99], np.nan, inplace=True)
    
    df.educ.replace([97, 98, 99], np.nan, inplace=True)
    df.maeduc.replace([97, 98, 99], np.nan, inplace=True)
    df.paeduc.replace([97, 98, 99], np.nan, inplace=True)
    df.speduc.replace([97, 98, 99], np.nan, inplace=True)
    
    df.income.replace([0, 13, 98, 99], np.nan, inplace=True)
    df.rincome.replace([0, 13, 98, 99], np.nan, inplace=True)    

In [22]:
gss_replace_invalid(gss)

In [23]:
# try to make the dataset smaller by replacing
# 64-bit FP numbers with 32-bit.

for varname in gss.columns:
    if gss[varname].dtype == np.float64:
        gss[varname] = gss[varname].astype(np.float32)

At this point, I have only moderate confidence that this code is correct.  I'm not sure I have dealt with every variable in the dataset, and I'm not sure that the special values for every variable are correct.

So I will ask for your help.

**Exercise**: In order to validate the other variables, I'd like each person who works with this notebook to validate one variable.

If you run the following cell, it will choose one of the columns from the dataset at random.  That's the variable you will check.

If you get `year` or `id_`, run the cell again to get a different variable name.

In [24]:
np.random.seed(None)
np.random.choice(gss.columns)

'spkath'

Go back through the previous two sections of this notebook and replace `polviews` with your randomly chosen variable.  Then run the cells again and go to [this online survey to report the results](https://forms.gle/tmST8YCu4qLc414F7). 

Note: Not all questions were asked during all years.  If your variable doesn't have data for 1974 or 2018, you might have to choose different years.

## Resampling

The GSS uses stratified sampling, which means that some groups are deliberately oversampled to help with statistical validity.

As a result, each respondent has a sampling weight which is proportional to the number of people in the population they represent.

Before running any analysis, we can compensate for stratified sampling by "resampling", that is, by drawing a random sample from the dataset, where each respondent's chance of appearing in the sample is proportional to their sampling weight.

In [25]:
def resample_rows_weighted(df, column):
    """Resamples a DataFrame using probabilities proportional to given column.

    df: DataFrame
    column: string column name to use as weights

    returns: DataFrame
    """
    weights = df[column]
    sample = df.sample(n=len(df), replace=True, weights=weights)
    return sample

In [26]:
def resample_by_year(df, column):
    """Resample rows within each year.

    df: DataFrame
    column: string name of weight variable

    returns DataFrame
    """
    grouped = df.groupby('year')
    samples = [resample_rows_weighted(group, column)
               for _, group in grouped]
    sample = pd.concat(samples, ignore_index=True)
    return sample

In [27]:
np.random.seed(19)
sample = resample_by_year(gss, 'wtssall')

## Saving the results

I'll save the results to an HDF5 file, which is a binary format that makes it much faster to read the data back.

First I'll save the original (not resampled) data.

An HDF5 file is like a dictionary on disk.  It contains keys and corresponding values.

`to_hdf` takes three arguments:

* The filename, `gss_eda.hdf5`.

* The key, `gss`

* The compression level, which controls how hard the algorithm works to compress the file.

So this file contains a single key, `gss`, which maps to the DataFrame with the original GSS data.

The argument `w` says that if the file already exists, we should overwrite it.

With compression level `3`, it reduces the size of the file by a factor of 10.

In [28]:
# save the original

gss.to_hdf('gss_eda.hdf5', 'gss', 'w', complevel=3)

In [29]:
!ls -l gss_eda.hdf5

-rw-rw-r-- 1 downey downey 6939805 Apr 17 11:51 gss_eda.hdf5


And I'll create a second file with three random resamplings of the original dataset.

In [30]:
# if the file already exists, remove it
import os

if os.path.isfile('gss_eda.3.hdf5'):
    !rm gss_eda.3.hdf5

This file contains three keys, `gss0`, `gss1`, and `gss2`, which map to three DataFrames.

In [31]:
# generate and store three resamplings
keys = ['gss0', 'gss1', 'gss2']

for i in range(3):
    np.random.seed(i)
    sample = resample_by_year(gss, 'wtssall')

    sample.to_hdf('gss_eda.3.hdf5', keys[i], complevel=3)

In [32]:
!ls -l gss_eda.3.hdf5

-rw-rw-r-- 1 downey downey 20717248 Apr 17 11:51 gss_eda.3.hdf5


For the other notebooks in this case study, we'll load this resampled data rather than reading and cleaning the data every time.